In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "" # username from the json file
os.environ['KAGGLE_KEY'] = "" # key from the json file
!kaggle datasets download -d andrewmvd/leukemia-classification

 99% 857M/867M [00:09<00:00, 91.2MB/s]
100% 867M/867M [00:09<00:00, 94.1MB/s]


In [2]:
!unzip "/content/leukemia-classification" -d "/content"

Streaming output truncated to the last 5000 lines.
  inflating: /content/C-NMC_Leukemia/training_data/fold_2/all/UID_25_12_5_all.bmp  
  inflating: /content/C-NMC_Leukemia/training_data/fold_2/all/UID_25_12_6_all.bmp  
  inflating: /content/C-NMC_Leukemia/training_data/fold_2/all/UID_25_12_7_all.bmp  
  inflating: /content/C-NMC_Leukemia/training_data/fold_2/all/UID_25_13_1_all.bmp  
  inflating: /content/C-NMC_Leukemia/training_data/fold_2/all/UID_25_13_2_all.bmp  
  inflating: /content/C-NMC_Leukemia/training_data/fold_2/all/UID_25_13_3_all.bmp  
  inflating: /content/C-NMC_Leukemia/training_data/fold_2/all/UID_25_14_1_all.bmp  
  inflating: /content/C-NMC_Leukemia/training_data/fold_2/all/UID_25_14_2_all.bmp  
  inflating: /content/C-NMC_Leukemia/training_data/fold_2/all/UID_25_14_3_all.bmp  
  inflating: /content/C-NMC_Leukemia/training_data/fold_2/all/UID_25_14_4_all.bmp  
  inflating: /content/C-NMC_Leukemia/training_data/fold_2/all/UID_25_14_5_all.bmp  
  inflating: /content/C-N

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
import os
import tqdm
import skimage.io
import glob

from tqdm import tqdm

from skimage.io import imread, imshow
from skimage.transform import resize

from sklearn.utils import shuffle

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import InputLayer, Conv2D, BatchNormalization, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications.xception import Xception, preprocess_input

%matplotlib inline

In [4]:
# Reading Data

train_dataset_0_all = glob.glob('C-NMC_Leukemia/training_data/fold_0/all/*.bmp')
train_dataset_0_hem = glob.glob('C-NMC_Leukemia/training_data/fold_0/hem/*.bmp')
train_dataset_1_all = glob.glob('C-NMC_Leukemia/training_data/fold_1/all/*.bmp')
train_dataset_1_hem = glob.glob('C-NMC_Leukemia/training_data/fold_1/hem/*.bmp')
train_dataset_2_all = glob.glob('C-NMC_Leukemia/training_data/fold_2/all/*.bmp')
train_dataset_2_hem = glob.glob('C-NMC_Leukemia/training_data/fold_2/hem/*.bmp')

#test_dataset  = glob.glob('../input/leukemia-classification/C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/*.bmp')
#valid_dataset = glob.glob('../input/leukemia-classification/C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data/*.bmp')

valid_data    = pd.read_csv('C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data_labels.csv')

In [5]:
A = []
H = []

A.extend(train_dataset_0_all)
A.extend(train_dataset_1_all)
A.extend(train_dataset_2_all)

H.extend(train_dataset_0_hem)
H.extend(train_dataset_1_hem)
H.extend(train_dataset_2_hem)

A = np.array(A)
H = np.array(H)

len(A),len(H)

(7272, 3389)

In [6]:
Image = []
Label = []

for i in tqdm(range(0, len(A))):
    img = imread(A[i])
    img = resize(img, (128,128))
    Image.append(img)
    Label.append(1)
    
for i in tqdm(range(0, len(H))):
    img = imread(H[i])
    img = resize(img, (128,128))
    Image.append(img)
    Label.append(0)
    
Image = np.array(Image)
Label = np.array(Label)

Image.shape, Label.shape

100%|██████████| 3389/3389 [01:22<00:00, 41.24it/s]


((10661, 128, 128, 3), (10661,))

In [7]:
Image, Label = shuffle(Image, Label, random_state = 42)

In [10]:
X = Image
y = Label
del Image 
del Label
del A
del H

In [12]:
valid_data.labels.value_counts()

1    1219
0     648
Name: labels, dtype: int64

In [13]:
X_val = []
for image_name in valid_data.new_names:
    # Loading images
    img = imread('C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data/' + image_name)
    # Resizing 
    img = resize(img, (128,128))
    # Appending them into list
    X_val.append(img)
 
# Converting into array
X_val = np.array(X_val)

# Storing target values as well 
y_val = valid_data.labels.values

In [14]:
# Augmentation & Applying preprocessing function of pre-trained model.

train_datagen  = ImageDataGenerator(horizontal_flip=True,
                                    vertical_flip=True,
                                    zoom_range = 0.2,
                                    preprocessing_function=preprocess_input)
train_datagen.fit(X)

In [15]:
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

valid_datagen.fit(X_val)

In [16]:
model = Xception(include_top=False, weights='imagenet', input_shape=(128,128,3))

In [17]:
for layers in model.layers:
    layers.trainable = False

In [18]:
x = Flatten()(model.output)


fcc_layer_1 = Dense(units = 1024, activation = 'relu')(x)
dropout_1   = Dropout(0.3)(fcc_layer_1)

fcc_layer_2 = Dense(units = 512, activation = 'relu')(dropout_1)
dropout_2   = Dropout(0.3)(fcc_layer_2)

final_layer = Dense(units = 1, activation = 'sigmoid')(dropout_2)

In [19]:
model = Model(inputs = model.input, outputs = final_layer)

In [20]:
model.compile(optimizer = 'adam', 
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [21]:
filepath = 'best_weights.hdf5'

earlystopping = EarlyStopping(monitor = 'val_accuracy', 
                              mode = 'max' , 
                              patience = 15,
                              verbose = 1)

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_accuracy', 
                                mode='max', 
                                save_best_only=True, 
                                verbose = 1)

learning_rate = ReduceLROnPlateau(monitor = 'val_accuracy',
                                  mode = 'max',
                                  patience = 5,
                                  factor = 0.3,
                                  min_delta = 0.00001)

callback_list = [earlystopping, learning_rate]

In [22]:
history = model.fit(train_datagen.flow(X, y, batch_size=512), 
                          validation_data = (X_val, y_val),
                          epochs = 5,
                          verbose = 1,
                          callbacks = callback_list)

Epoch 1/5
21/21 [==============================] - 81s 2s/step - loss: 0.6441 - accuracy: 0.6649 - val_loss: 0.8140 - val_accuracy: 0.6529
Epoch 2/5
21/21 [==============================] - 40s 2s/step - loss: 0.6175 - accuracy: 0.6871 - val_loss: 0.7191 - val_accuracy: 0.6529
Epoch 3/5
21/21 [==============================] - 40s 2s/step - loss: 0.6003 - accuracy: 0.6832 - val_loss: 0.7270 - val_accuracy: 0.6417
Epoch 4/5
21/21 [==============================] - 41s 2s/step - loss: 0.5532 - accuracy: 0.7308 - val_loss: 1.2053 - val_accuracy: 0.5613
Epoch 5/5
21/21 [==============================] - 40s 2s/step - loss: 0.5373 - accuracy: 0.7392 - val_loss: 1.3263 - val_accuracy: 0.5886


In [28]:
from tensorflow.keras.preprocessing import image

data = image.load_img('/content/Leukemia.png', target_size=(128, 128, 3))
data = image.img_to_array(data)
data = np.expand_dims(data, axis=0)
model.predict(data)[0][0]

0.0023055277

In [29]:
model.save('model4.h5')